In [1]:
from array import array
import subprocess
import math
import ROOT
import os
import numpy as np

Welcome to JupyROOT 6.29/01


In [2]:
buildcuda = '/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/'

In [3]:
buildoneapi = '/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/'

In [4]:
buildacpp = '/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/'

In [5]:
!ls /home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx

CMakeFiles	     InvariantMasses	  SInvariantMasses
cmake_install.cmake  InvariantMassesSYCL  SInvariantMassesSYCL


In [6]:
!ls /home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx

CMakeFiles	     InvariantMasses	  SInvariantMasses
cmake_install.cmake  InvariantMassesSYCL  SInvariantMassesSYCL


In [7]:
!echo $LD_LIBRARY_PATH

/home/mdessole/Projects/AdaptiveCpp/AdaptiveCpp_install/lib:/usr/local/cuda-11.8/lib64:/home/mdessole/Projects/ROOT/root/lib:/opt/rocm-5.7.0/lib:/opt/rocm-5.7.0/opencl/lib:/home/mdessole/Projects/OneAPI/build/lib


In [8]:
sizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [9]:
Ssizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000,200000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [10]:
def result2list(exe, N, nruns):
    print(exe, str(int(N)), str(int(nruns)))
    result = subprocess.run([exe, str(int(N)), str(int(nruns)) ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)
    if len(result)<=0:
        return []
    result = result.replace("\r","").replace("\n","")
    time = np.zeros(nruns)
    for r in range(nruns):
        i = result.find("time")
        j = result.find("(s)")
        if (i<0) or (j<0):
            break
        time[r] = float(result[i+5:j])
        result = result[j+3:]
    #endif
    return time

def run_benchmark(builddir, sizes, nruns = 1, testname = "InvariantMasses", btype = 'cpu'):
    timeopt = array( 'd' )
    timestd = array( 'd' )
    
    if btype == 'cpu':
        exe = os.path.join(builddir,'testx',testname)
    elif btype == 'cuda':
        exe = os.path.join(builddir,'testx',testname+'CUDA')
    elif btype == 'sycl':
        exe = os.path.join(builddir,'testx',testname+'SYCL')
    #endif
    
    if (not os.path.exists(exe)):
        print("Executable", exe, "not found!")
        return timeopt, timestd
    
    for i, N in enumerate(sizes):
        time = result2list(exe,N,nruns)
        if (len(time)<=0):
            return timeopt, timestd
        print(time, '\n')
        timeopt.append(np.amin(time))
        timestd.append(np.std(time)) 
    #endfor
    return timeopt, timestd

In [11]:
timecpu, stdcpu = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 1000 3
RVec cpu time 6e-05 (s)
RVec cpu time 6.1e-05 (s)
RVec cpu time 6e-05 (s)

[6.0e-05 6.1e-05 6.0e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 10000 3
RVec cpu time 0.000612 (s)
RVec cpu time 0.000608 (s)
RVec cpu time 0.000611 (s)

[0.000612 0.000608 0.000611] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 100000 3
RVec cpu time 0.006106 (s)
RVec cpu time 0.006189 (s)
RVec cpu time 0.006106 (s)

[0.006106 0.006189 0.006106] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 1000000 3
RVec cpu time 0.064191 (s)
RVec cpu time 0.061164 (s)
RVec cpu time 0.060647 (s)

[0.064191 0.061164 0.060647] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMasses 5000000 3
RVec cpu time 0.325073 (s)
RVec cpu time 0.327538 (s)
RVec cpu time 0.325127 (s)

[0.325073 0.327538 0.325127] 

/home/

In [12]:
timecpu

array('d', [6e-05, 0.000608, 0.006106, 0.060647, 0.325073, 0.640106, 3.23721, 6.4347])

In [13]:
Stimecpu, Sstdcpu = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'cpu')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 1000 3
SINGLE_PRECISION defined 
RVec cpu time 6.1e-05 (s)
RVec cpu time 4.5e-05 (s)
RVec cpu time 4.5e-05 (s)

[6.1e-05 4.5e-05 4.5e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 10000 3
SINGLE_PRECISION defined 
RVec cpu time 0.000449 (s)
RVec cpu time 0.000445 (s)
RVec cpu time 0.000449 (s)

[0.000449 0.000445 0.000449] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 100000 3
SINGLE_PRECISION defined 
RVec cpu time 0.004462 (s)
RVec cpu time 0.004489 (s)
RVec cpu time 0.004495 (s)

[0.004462 0.004489 0.004495] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 1000000 3
SINGLE_PRECISION defined 
RVec cpu time 0.046738 (s)
RVec cpu time 0.04435 (s)
RVec cpu time 0.044239 (s)

[0.046738 0.04435  0.044239] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMasses 5000000 3
SINGLE_

In [14]:
timeoneapi, stdoneapi = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.00123 (s)
sycl time 0.000329 (s)
sycl time 0.000231 (s)

[0.00123  0.000329 0.000231] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000949 (s)
sycl time 0.000413 (s)
sycl time 0.000366 (s)

[0.000949 0.000413 0.000366] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.002849 (s)
sycl time 0.002371 (s)
sycl time 0.002226 (s)

[0.002849 0.002371 0.002226] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/InvariantMassesSYCL 1000000 3

In [15]:
timeoneapi

array('d', [0.000231, 0.000366, 0.002226, 0.018255, 0.081658, 0.159471, 0.758867, 1.51382])

In [16]:
Stimeoneapi, Sstdoneapi = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000777 (s)
sycl time 0.000384 (s)
sycl time 0.000218 (s)

[0.000777 0.000384 0.000218] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000748 (s)
sycl time 0.000397 (s)
sycl time 0.00026 (s)

[0.000748 0.000397 0.00026 ] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_oneapi/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001601 (s)
sycl time 0.001078 (s)
sycl time 0.001025 (s)

[0.001601 0.001078 0.001025] 

/home/mdess

In [17]:
timeacpp, stdacpp = run_benchmark(buildacpp, sizes,  nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/InvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001092 (s)
sycl time 0.000442 (s)
sycl time 0.000437 (s)

[0.001092 0.000442 0.000437] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/InvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000851 (s)
sycl time 0.000463 (s)
sycl time 0.000431 (s)

[0.000851 0.000463 0.000431] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/InvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.002634 (s)
sycl time 0.00227 (s)
sycl time 0.002379 (s)

[0.002634 0.00227  0.002379] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/Invariant

In [18]:
timeacpp

array('d', [0.000437, 0.000431, 0.00227, 0.016886, 0.079907, 0.155481, 0.755182, 1.52333])

In [19]:
Stimeacpp, Sstdacpp = run_benchmark(buildacpp, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/SInvariantMassesSYCL 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001188 (s)
sycl time 0.000283 (s)
sycl time 0.000311 (s)

[0.001188 0.000283 0.000311] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/SInvariantMassesSYCL 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000835 (s)
sycl time 0.000469 (s)
sycl time 0.000372 (s)

[0.000835 0.000469 0.000372] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_adaptivecpp/testx/SInvariantMassesSYCL 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001544 (s)
sycl time 0.001056 (s)
sycl time 0.001073 (s)

[0.001544 0.001056 0.00107

In [20]:
timecuda, stdcuda = run_benchmark(buildcuda, sizes, nruns = 3, btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000386 (s)
cuda time 4.5e-05 (s)
cuda time 4.3e-05 (s)

[3.86e-04 4.50e-05 4.30e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000421 (s)
cuda time 0.000177 (s)
cuda time 0.000172 (s)

[0.000421 0.000177 0.000172] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.00176 (s)
cuda time 0.001416 (s)
cuda time 0.001363 (s)

[0.00176  0.001416 0.001363] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/InvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.013152 (s)
cuda time 0.012

In [21]:
timecuda

array('d', [4.3e-05, 0.000172, 0.001363, 0.012489, 0.060647, 0.113511, 0.567637, 1.15574])

In [22]:
timecuda

array('d', [4.3e-05, 0.000172, 0.001363, 0.012489, 0.060647, 0.113511, 0.567637, 1.15574])

In [23]:
Stimecuda, Sstdcuda = run_benchmark(buildcuda, Ssizes, nruns = 3, testname = "SInvariantMasses", btype = 'cuda')

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 1000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000358 (s)
cuda time 2.9e-05 (s)
cuda time 2.8e-05 (s)

[3.58e-04 2.90e-05 2.80e-05] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 10000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.0003 (s)
cuda time 9.4e-05 (s)
cuda time 0.000109 (s)

[3.00e-04 9.40e-05 1.09e-04] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 100000 3
ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000827 (s)
cuda time 0.000639 (s)
cuda time 0.000576 (s)

[0.000827 0.000639 0.000576] 

/home/mdessole/Projects/ROOT/genvectorxfork/build_cuda/testx/SInvariantMassesCUDA 1000000 3
ROOT_MEAS_TIMING defined 
ROOT_MAT

In [29]:
n = len(sizes) 
speedupacpp = array('d', [timecpu[i]/timeacpp[i] for i in range(n)])
speeduponeapi   = array('d', [timecpu[i]/timeoneapi[i] for i in range(n)])
speedupcuda     = array('d', [timecpu[i]/timecuda[i] for i in range(n)])

In [30]:
n = len(Ssizes)
Sspeedupacpp = array('d', [Stimecpu[i]/Stimeacpp[i] for i in range(n)])
Sspeeduponeapi   = array('d', [Stimecpu[i]/Stimeoneapi[i] for i in range(n)])
Sspeedupcuda     = array('d', [Stimecpu[i]/Stimecuda[i] for i in range(n)])

In [25]:
print(Sspeedupacpp)
print(Sspeeduponeapi)
print(Sspeedupcuda)

array('d', [0.1590106007067138, 1.196236559139785, 4.225378787878787, 6.281272185148374, 7.551374911864624, 7.174963802717107, 7.908544410672071, 8.026858860272986, 8.108118943038416])
array('d', [0.20642201834862386, 1.7115384615384617, 4.3531707317073165, 6.07595110561736, 7.084551085453123, 7.217542881048668, 7.85814717499052, 7.878552554440639, 8.02911808618056])
array('d', [1.6071428571428572, 4.73404255319149, 7.746527777777778, 9.19920981493034, 10.10975714408307, 9.92316049593115, 10.408060766875119, 10.27570860880184, 10.546973219862851])


In [26]:
val = list(sizes)
sizes_gb = [v*8*4/1e9 for v in val] #(nb of doubles)*(bytes for doubles)*(4=dim of LVector)/(bytes in GB)
sizes_gb = array('d', sizes_gb)
sizes_gb

array('d', [3.2e-05, 0.00032, 0.0032, 0.032, 0.16, 0.32, 1.6, 3.2])

In [27]:
val = list(Ssizes)
Ssizes_gb = [v*4*4/1e9 for v in val] #(nb of floats)*(bytes for floats)*(4=dim of LVector)/(bytes in GB)
Ssizes_gb = array('d', Ssizes_gb)
Ssizes_gb

array('d', [1.6e-05, 0.00016, 0.0016, 0.016, 0.08, 0.16, 0.8, 1.6, 3.2])

In [31]:
p = ROOT.TGraph(len(sizes), sizes, speedupacpp)
g = ROOT.TGraph(len(sizes), sizes, speeduponeapi)
q = ROOT.TGraph(len(sizes), sizes, speedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec")
q.GetXaxis().SetTitle( 'Input size (# doubles)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"AdaptiveCpp","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("speedup.png")

Info in <TCanvas::Print>: png file speedup.png has been created


In [30]:
len(Ssizes), Ssizes, len(Sspeedupopensycl)

(9,
 array('d', [1000.0, 10000.0, 100000.0, 1000000.0, 5000000.0, 10000000.0, 50000000.0, 100000000.0, 200000000.0]),
 9)

In [32]:
p = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupacpp)
g = ROOT.TGraph(len(Ssizes), Ssizes, Sspeeduponeapi)
q = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec - Single Precision")
q.GetXaxis().SetTitle( 'Input size (# floats)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"AdaptiveCpp","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sspeedup.png")

Info in <TCanvas::Print>: png file Sspeedup.png has been created


In [33]:
p = ROOT.TGraphErrors(len(sizes), sizes, timeacpp, stdacpp)
g = ROOT.TGraphErrors(len(sizes), sizes, timeoneapi, stdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, timecuda, stdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, timecpu, stdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times")
r.GetXaxis().SetTitle( 'Input size (# doubles)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"AdaptiveCpp","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("exectimes.png")

Info in <TCanvas::Print>: png file exectimes.png has been created


In [34]:
p = ROOT.TGraphErrors(len(sizes), sizes, Stimeacpp, Sstdacpp)
g = ROOT.TGraphErrors(len(sizes), sizes, Stimeoneapi, Sstdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, Stimecuda, Sstdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, Stimecpu, Sstdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times - Single Precision")
r.GetXaxis().SetTitle( 'Input size (# floats)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"AdaptiveCpp","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sexectimes.png")

Info in <TCanvas::Print>: png file Sexectimes.png has been created
